In [1]:
#IMPORT STATEMENTS
import selenium
import numpy as np
import warnings
warnings.filterwarnings('ignore')  # supress scikit 'future warnings'
import pandas as pd
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from scipy.optimize import linear_sum_assignment
import matplotlib         
from matplotlib import pyplot as plt
from scipy.stats import norm
from scipy.stats import norm, kurtosis
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import svm
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.interpolate import make_interp_spline, BSpline
from scipy import stats
import math
import pickle
import random
import copy
import itertools
from dataclasses import dataclass
random.seed(42)
import torch
import torch.nn as nn
#from basketball_reference_scraper import ask_matches
from basketball_reference_scraper.teams import get_roster, get_team_stats, get_opp_stats, get_roster_stats, get_team_misc
from basketball_reference_scraper.players import get_stats, get_game_logs, get_player_headshot, get_player_suffix, lookup

In [2]:
# Read in the list of all courses
df = pd.read_excel ('export.xls')

# Drop unneeded columns and categories
df = df.drop(['Title', 'Course ID', 'Catalog Print', 'Offer Number', 'Tags'], axis=1)

# Use a dictionary
course_start_dict = {}
rows = len(df.index)

# Loop through all the rows in the dataframe
for row in range(rows):
    subject_area = df.loc[row]["Subject"]
    if course_start_dict.get(subject_area, -1) == -1:
        course_start_dict[subject_area] = row

# Sanity Check print statement
df.loc[0]

Subject            AA
Course Number    47SI
Name: 0, dtype: object

In [3]:
# Convert an input depending on how it is inputted
def convert_input(input_course):
    # Split based on space if it exists
    pieces_list = input_course.split()
    
    # If there are two pieces from splitting return
    # the two pieces
    if len(pieces_list) == 2:    
        return pieces_list[0], pieces_list[1]
    
    # Use a split index to find where department changes to number
    split_index = -1
    for index in range(len(input_course)):
        if input_course[index].isdigit():
            split_index = index
            break
    
    # Return the the two parts if using a split_index
    return input_course[:split_index], input_course[split_index:]
    
# Split number function -> Takes in course number and 
# returns tuple of number with any letter after
# Example: 43S -> (43, 'S')
def split_number(course_num):
        index = 1 # Start index at 1

        # Continue to iterate index until end of digits
        while index <= len(course_num) and course_num[:index].isdigit():
            index += 1

        # Return here if no additional characters after number
        return (course_num[:index-1], course_num[index-1:])

        # Continue to loop in case of characters
        index = len(course_num) - 1
        while index >= 0:
            if not(chr(course_num[index]).isdigt()):
                while index >= 0 and not(chr(course_num[index]).isdigit()):
                    index -= 1

                return (course_num[:index+1], course_num[index+1:])

            index -= 1

        # Return only course number with no letter
        return (course_num, "")
    
# Full new course recommender based on input_course
def recommend_new_course(input_course):
    # Get the subject area and the course number by converting given input
    subject_area, course_number = convert_input(input_course)
    
    # Get the pure number by splitting the number
    course_number_pure = split_number(course_number)[0]
    
    # Get the row that this is found
    row_found = df.loc[(df['Subject'] == subject_area) & (df['Course Number'] == course_number)]
    
    # Ensure no bugs
    assert len(row_found.index) == 1
    
    # This is the recommendation row to start at before loooping up
    rec_row = row_found.index[0]
    
    # Continue to increase the recommendation row (of the course)
    # while the conditions are true (checking for repeats or sections)
    while (rec_row <= rows and df.loc[rec_row]['Subject'] == subject_area and
           ((df.loc[rec_row]['Course Number'] == course_number) or
           (split_number(df.loc[rec_row]['Course Number'])[0] == course_number) or 
           (split_number(df.loc[rec_row]['Course Number'])[0] == course_number_pure and 
           split_number(df.loc[rec_row]['Course Number'])[1] != "A" and 
           split_number(df.loc[rec_row]['Course Number'])[1] != "B" and
           split_number(df.loc[rec_row]['Course Number'])[1] != "C")
          )):
        rec_row += 1
        
    # If we go past the subject area (disertation), bump down back to same
    # department with the dissertation again
    if df.loc[rec_row]['Subject'] != subject_area:
        rec_row -= 1
       
    # Make the recommendation by concatenating the different found components
    recommendation = str(df.loc[rec_row]['Subject']) + " " + str(df.loc[rec_row]['Course Number'])
    return recommendation
    
recommend_new_course("CS 106B")    
    

'CS 107'